In [2]:
import numpy as np
import pandas as pd
import time
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList, cuda
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chaineripy.extensions import PrintReport, ProgressBar

In [4]:
def zs(p, n, shift=0):
    return (p.shift(shift) - p.rolling(n).mean()) / p.rolling(n).std()

def avg(p, n):
    return p.rolling(n).mean()

def std(p, n):
    return p.rolling(n).std()

In [10]:
start = time.time()
print('loading from db')
df = pd.read_sql('select * from history15min', session.bind)
print('loaded from db: %fs' % (time.time() - start))
df.exec_date = pd.to_datetime(df.exec_date)
df = df.set_index('exec_date')
df = df.loc['2016-03':]
# df = df[['id', 'side', 'price', 'size']]

loading from db
loaded from db: 1.667845s


In [11]:
df

,price,size
exec_date,,
2016-03-01 00:00:00,4.994609e+04,25.987227
2016-03-01 00:15:00,4.998313e+04,15.549000
2016-03-01 00:30:00,5.001167e+04,51.568395
2016-03-01 00:45:00,5.011319e+04,4.485000
2016-03-01 01:00:00,4.989262e+04,30.907859
2016-03-01 01:15:00,4.992312e+04,16.026564
2016-03-01 01:30:00,4.991145e+04,6.549000
2016-03-01 01:45:00,4.989500e+04,5.048000
2016-03-01 02:00:00,4.989188e+04,34.441220


In [24]:
dfb = df

p = dfb.price
v = dfb['size']

dfb['pma12'] = zs(p / avg(p, 12) - 1, 96)
dfb['pma96'] = zs(p / avg(p, 96) - 1, 96)
dfb['pma672'] = zs(p / avg(p, 672) - 1, 96)

dfb['ma4_36'] = zs(avg(p, 4) / avg(p, 36) - 1, 96)
dfb['ma12_96'] = zs(avg(p, 12) / avg(p, 96) - 1, 96)
dfb['ac12_12'] = zs((p / avg(p, 12)) / avg(p / avg(p, 12), 12), 96)
dfb['ac96_96'] = zs((p / avg(p, 96)) / avg(p / avg(p, 96), 12), 96)

dfb['vma12'] = zs(v / avg(v, 12) - 1, 96)
dfb['vma96'] = zs(v / avg(v, 96) - 1, 96)
dfb['vma672'] = zs(v / avg(v, 672) - 1, 96)

dfb['vZ12'] = zs(v, 12)
dfb['vZ96'] = zs(v, 96)
dfb['vZ672'] = zs(v, 672)

r_label_list = [
    'r', 'r_1', 'r_2',
]
for r_label in r_label_list:
    dfb[r_label] = 0
dfb['state'] = 0

dfb['pZ12'] = zs(p, 12)
dfb['pZ96'] = zs(p, 96)
dfb['vol12'] = zs(std(p, 12), 96)
dfb['vol96'] = zs(std(p, 96), 96)
dfb['vol672'] = zs(std(p, 672), 96)
dfb['dv12_96'] = zs(std(p, 12) / avg(std(p, 12), 96), 96)
dfb['dv96_672'] = zs(std(p, 96) / avg(std(p, 96), 672), 96)
for i in range(96):
    dfb['pZ96_s%02d' % i] = zs(p, 96, shift=i)

dfb['utctime'] = (dfb.index.hour * 4 + dfb.index.minute / 15) / 96

dfz = dfb

indicator = dfz.reset_index().loc[:, 'pma12':'utctime']
price_history = dfz.reset_index().price

indicator['vma12'] = indicator.vma12.fillna(0)
indicator['vZ12'] = indicator.vZ12.fillna(0)

In [13]:
answer = price_history.shift(-1) / price_history - 1
answer = pd.qcut(answer, 3, labels=list(range(3))).fillna(1)
answer

0        2
1        2
2        2
3        0
4        2
5        1
6        1
7        1
8        0
9        2
10       1
11       2
12       0
13       1
14       1
15       0
16       1
17       1
18       1
19       0
20       2
21       1
22       0
23       2
24       0
25       2
26       1
27       2
28       0
29       0
        ..
65403    0
65404    2
65405    2
65406    2
65407    2
65408    2
65409    0
65410    0
65411    0
65412    2
65413    0
65414    0
65415    2
65416    0
65417    2
65418    0
65419    0
65420    0
65421    0
65422    2
65423    2
65424    0
65425    0
65426    2
65427    2
65428    1
65429    0
65430    2
65431    2
65432    1
Name: price, Length: 65433, dtype: category
Categories (3, int64): [0 < 1 < 2]

In [14]:
indexer = indicator.dv96_672.notnull()

In [15]:
indicator.loc[indexer].isnull().any().value_counts()

False    25
dtype: int64

In [16]:
answer[indexer].isnull().any()

False

In [17]:
xp = cuda.cupy

d_exp = indicator.loc[indexer]
d_obj = answer[indexer]
data = xp.array(d_exp, dtype=xp.float32)
t_data = xp.array(d_obj, dtype=xp.int32)

In [18]:
row, col = d_exp.shape

ls_1 = 800
ls_2 = 1000
ls_3 = 1000
ls_4 = 200
out_size = 3
class MyChain(Chain):
    def __init__(self):
        super().__init__(
            l1=L.Linear(col, ls_1),
            l2=L.Linear(ls_1, ls_2),
            l3=L.Linear(ls_2, ls_3),
            l4=L.Linear(ls_3, ls_4),
            l5=L.Linear(ls_4, out_size)
        )
    
    def __call__(self, x):
        h1 = F.sigmoid(self.l1(x))
        h2 = F.sigmoid(self.l2(h1))
        h3 = F.sigmoid(self.l3(h2))
        h4 = F.sigmoid(self.l4(h3))
        o = self.l5(h4)
        return o

In [19]:
train = datasets.tuple_dataset.TupleDataset(data, t_data)
train = iterators.SerialIterator(train, batch_size=100, shuffle=True, repeat=True)

In [20]:
model = L.Classifier(MyChain())

gpu_device = 0
cuda.get_device_from_id(gpu_device).use()
model.to_gpu(gpu_device)

optimizer = optimizers.SGD()
optimizer.setup(model)
updater = training.StandardUpdater(train, optimizer)

In [22]:
interval = 25
times = 16
for i in range(times):
    trainer = training.Trainer(updater, (interval * (i + 1), 'epoch'), out='result3')
    trainer.extend(extensions.LogReport())
    trainer.extend(PrintReport(['epoch', 'elapsed_time', 'main/loss', 'main/accuracy']))
    trainer.extend(ProgressBar(update_interval=10))
    trainer.run()
    model.to_cpu()
    serializers.save_npz('agent/snapshot3_%02d.npz' % i, model.predictor)
    model.to_gpu(gpu_device)

A Jupyter Widget

A Jupyter Widget

KeyboardInterrupt: 

In [25]:
dfb

,price,size,pma12,pma96,pma672,ma4_36,ma12_96,ac12_12,ac96_96,vma12,...,pZ96_s86,pZ96_s87,pZ96_s88,pZ96_s89,pZ96_s90,pZ96_s91,pZ96_s92,pZ96_s93,pZ96_s94,pZ96_s95
exec_date,,,,,,,,,,,,,,,,,,,,,
2016-03-01 00:00:00,4.994609e+04,25.987227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-01 00:15:00,4.998313e+04,15.549000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-01 00:30:00,5.001167e+04,51.568395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-01 00:45:00,5.011319e+04,4.485000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-01 01:00:00,4.989262e+04,30.907859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-01 01:15:00,4.992312e+04,16.026564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-01 01:30:00,4.991145e+04,6.549000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-01 01:45:00,4.989500e+04,5.048000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-01 02:00:00,4.989188e+04,34.441220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
import datetime

In [29]:
ts = dfb.index[-1]
ts

Timestamp('2018-01-11 14:00:00')

In [31]:
ts.to_pydatetime()

datetime.datetime(2018, 1, 11, 14, 0)